# PyTorch扩展
`作者：Tina`
`时间：2018-05-08`

这个笔记中，我们将讨论扩展[torch.nn](torch.nn)和[torch.autograd](https://pytorch.org/docs/stable/autograd.html#module-torch.autograd)的方法。
使用C库编写自定义的C扩展。

## 扩展torch.autograd
在`autograd`中添加操作需要为每个操作实现一个新的[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)子类。
`Function`是被`autograd`用来计算结果和梯度的，并对操作历史进行编码。

每个新函数都要求实现2个方法：
- [forward](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function.forward)，执行操作的代码。如果你指定默认值，它可以取任意数量的参数，其中一些是可选的。它可以接受各种各样的Python对象，`Variable`参数将在调用之前转换为`Tensor`，它们的使用将会被注册在图中。注意，这个逻辑不会遍历list/dic/任何其他数据结构，并且只考虑被调用的直接参数的变量。如果有多个输出的话，你可以返回一个Tensor，或一个Tensor元组。另外, 请参阅 Function文档来查找只能从`forward()`调用的方法的描述。
- [backward](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function.backward)，梯度公式。它会被赋予与输出一样多的变量参数，每一个都代表了关于那个输出的梯度。它应该返回和输入一样多的变量，每一个都包含了其相应的输入的梯度。如果输入不需要梯度(见`needs_input_grad`)，或者是非变量对象，则可以返回`None`。另外，如果对`forward()`有可选参数，则可以返回比输入更多的梯度，只要它们都是`None`。

下面是`torch.nn`模块的`Linear`函数代码, 以及注解:

```python
# 继承Function
class LinearFunction(Function):

    # 注意前向和后向都是静态方法 @staticmethods
    @staticmethod
    # bias是一个可选参数
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # 这个函数只有一个输出，所以它只有一个梯度
    @staticmethod
    def backward(ctx, grad_output):
        # 这是一种非常方便的模式 - 在反向传播的顶端
        # 取出saved_tensors，初始化所有输入的梯度为None
        # 由于额外的尾部Nones被忽略了，即使函数有可选的输入，return语句也很简单。
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # 这些needs_input_grad检查是可选的，只是为了提高效率。如果想让代码更简单,可以跳过他们。返回不需要的输入的梯度不会有错误
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias
```

现在，为了更方便地使用这些自定义操作，建议建议使用`apply`方法：
```python
linear = LinearFunction.apply
```
在这里，给出一个附加的例子，它是由非变量参数进行参数化的：
```python
class MulConstant(Function):
    @staticmethod
    def forward(ctx, tensor, constant):
        # ctx是一个上下文对象,可以用来储存信息
        # for backward computation
        ctx.constant = constant
        return tensor * constant

    @staticmethod
    def backward(ctx, grad_output):
        # 我们返回的输入梯度和参数一样多。
        # 非张量参数的梯度必须是None.
        return grad_output * ctx.constant, None
```

想要检查实现的反向传播方法是否计算了函数的导数，可以使用有限差分进行数值估计：
```python
from torch.autograd import gradcheck

# gradchek takes a tuple of tensor as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (Variable(torch.randn(20,20).double(), requires_grad=True), Variable(torch.randn(30,20).double(), requires_grad=True),)
test = gradcheck(Linear.apply, input, eps=1e-6, atol=1e-4)
print(test)
```

## 扩展torch.nn模块
`nn`模块有两种类型的接口 - 模块和它们函数的版本。可以通过这两种方式来扩展它，但是建议使用各种层的模块，这些层包含任何参数或缓冲区，而且推荐使用函数形式的无参数（parameter-less）操作，如激活函数、池化等。

上面的部分中已经完整介绍了一个添加操作的函数版本。

### 添加一个模块
由于`nn`大量利用了`autograd`，添加一个新模块需要实现一个执行操作并可以计算梯度的函数。假设我们想要实现一个`Linear`模块，并且已经在上面部分中实现了。添加这个模块的代码非常少。现在，有两个功能需要实现：
- `__init__`（可选），接受卷积核尺寸，特征数量等参数，并且初始化参数和缓冲区。
- `forward()`，实例化一个`Function`类，并且用它执行操作。

下面是一个`linear`模块的实现方式：

```python
class Linear(nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter是一种特殊的变量,一旦它被分配为一个属性，它就会自动注册为模块的参数。
        # 参数和缓冲区需要注册，否则它们不会出现在 .parameters() (doesn't apply to buffers)，并且，当例如.cuda()被调用时，不会被转换。
        # 可以使用.register_buffer()来注册缓冲区。
        # nn.Parameters需要默认梯度
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # 应该总是注册所有可能的参数，但如果你想，可选参数也可以是None 
            self.register_parameter('bias', None)

        # 这不是一个很聪明的初始化权重的方法
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # 请参阅autograd部分，就可以明白这里在做什么。
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)设置关于这个模块的额外信息。你可以通过打印这个类的对象来测试它。
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
```

## 编写自定义C扩展
现在你可以在[这里](https://github.com/pytorch/extension-ffi)找到一个例子